In [50]:
import numpy as np
import random
import math
from collections import defaultdict

In [51]:
capacity = 20
max_movement = 7
max_value_iterations = 10000

lambda_demand1, lambda_demand2, lambda_return1, lambda_return2 = 3, 4, 3, 2
gamma = 0.9
actions = list(range(-max_movement, max_movement + 1))
theta = 0.01
" ".join([str(x) for x in actions])

'-7 -6 -5 -4 -3 -2 -1 0 1 2 3 4 5 6 7'

In [52]:
def is_feasible(s, a, car_capacity):
    if a > 0 and (s[0] < a):
        return False

    if a < 0 and (s[1] < -a):
        return False

    return True

In [53]:
def poisson(lam, n):
    l_pois = [math.exp(-lam) * (lam ** k) / math.factorial(k) for k in range(n)]
    l_pois.append(1 - sum(l_pois))
    return l_pois

def initialization(car_capacity):
     v = 100 * np.random.uniform(size = [car_capacity + 1, car_capacity + 1])
     return v

In [54]:
def new_value(v, s, a, gamma, car_capacity, l_prob_demand1, l_prob_demand2, l_prob_return1, l_prob_return2):
    if not is_feasible(s, a, car_capacity):
        raise Exception("N/A")

    p_next_states1, p_rented1 = np.zeros(car_capacity+1), np.zeros(car_capacity+1)

    s_0 = min(car_capacity,max(s[0] - a,0))
    s_1 = min(car_capacity,max(s[1] + a,0))


    for demand1 in range(car_capacity + 1):
        rented1 = int(min(s_0,demand1))
        not_rented1 = s_0 - rented1
        prob_demand1 = l_prob_demand1[demand1]
        for return1 in range(car_capacity + 1):
            next_state1 = int(min(not_rented1 + return1, car_capacity))
            p_joint = prob_demand1 * l_prob_return1[return1]
            p_next_states1[next_state1] += p_joint
            p_rented1[rented1] += p_joint

    p_next_states2, p_rented2 = np.zeros(car_capacity+1), np.zeros(car_capacity + 1)

    for demand2 in range(car_capacity + 1):
        rented2 = int(min(s_1,demand2))
        not_rented_2 = s_1 - rented2
        p_demand2 = l_prob_demand2[demand2]
        for return_2 in range(car_capacity + 1):
            next_state2 = int(min(not_rented_2 + return_2, car_capacity))
            p_joint = p_demand2 * l_prob_return2[return_2]
            p_next_states2[next_state2] += p_joint
            p_rented2[rented2] += p_joint

    result = -2 * abs(a)
    for state_1 in range(car_capacity + 1):
        for state_2 in range(car_capacity + 1):
            result += p_next_states1[state_1]*p_next_states2[state_2] * gamma * v[state_1,state_2]

    for rented1 in range(car_capacity + 1):
        result += p_rented1[rented1]*rented1 * 10

    for rented2 in range(car_capacity + 1):
        result += p_rented2[rented2]*rented2 * 10

    return result

In [55]:
v = initialization(capacity)
l_prob_demand1 = poisson(lambda_demand1, capacity)
l_prob_demand_2 = poisson(lambda_demand2, capacity)
l_prob_return1 = poisson(lambda_return1, capacity)
l_prob_return2 = poisson(lambda_return2, capacity)
for i,prob_demand in zip(range(len(l_prob_demand1)), l_prob_demand1):
    print("{} : {:.15f}".format(i, prob_demand))
print(sum(l_prob_demand1))

0 : 0.049787068367864
1 : 0.149361205103592
2 : 0.224041807655388
3 : 0.224041807655388
4 : 0.168031355741541
5 : 0.100818813444924
6 : 0.050409406722462
7 : 0.021604031452484
8 : 0.008101511794681
9 : 0.002700503931560
10 : 0.000810151179468
11 : 0.000220950321673
12 : 0.000055237580418
13 : 0.000012747133943
14 : 0.000002731528702
15 : 0.000000546305740
16 : 0.000000102432326
17 : 0.000000018076293
18 : 0.000000003012715
19 : 0.000000000475692
20 : 0.000000000083144
1.0


In [56]:
import datetime

l_states1, l_states2 = list(range(capacity + 1)), list(range(capacity + 1))

for iter_no in range(1,max_value_iterations + 1):
    delta = 0
    for state_1 in l_states1:
        for state_2 in l_states2:
            s = np.array([state_1,state_2])
            max_value = -math.inf
            for a in actions:
                if not is_feasible(s, a, capacity):
                    continue
                v_s_a = new_value(v, s, a, gamma, capacity, l_prob_demand1, l_prob_demand_2, l_prob_return1, l_prob_return2)
                if v_s_a > max_value:
                    max_value, best_action = v_s_a, a
            delta = max(delta, abs(v[state_1,state_2]-max_value))
            v[state_1,state_2] = max_value

    print(datetime.datetime.now(),"iteration & value:",str(iter_no).rjust(3),"{:.5f}".format(delta).rjust(8))
    if delta < theta:
        break

2026-01-17 19:43:50.307535 iteration & value:   1 310.03382
2026-01-17 19:43:54.201760 iteration & value:   2 113.94864
2026-01-17 19:43:58.340539 iteration & value:   3 80.56098
2026-01-17 19:44:02.138860 iteration & value:   4 57.64667
2026-01-17 19:44:06.192980 iteration & value:   5 43.22109
2026-01-17 19:44:10.251458 iteration & value:   6 34.20603
2026-01-17 19:44:14.182149 iteration & value:   7 28.26737
2026-01-17 19:44:18.175569 iteration & value:   8 23.25364
2026-01-17 19:44:22.177209 iteration & value:   9 19.08463
2026-01-17 19:44:26.028560 iteration & value:  10 15.64302
2026-01-17 19:44:30.035445 iteration & value:  11 12.81333
2026-01-17 19:44:34.141863 iteration & value:  12 10.49331
2026-01-17 19:44:38.048551 iteration & value:  13  8.59226
2026-01-17 19:44:41.879428 iteration & value:  14  7.03476
2026-01-17 19:44:45.692666 iteration & value:  15  5.75970
2026-01-17 19:44:49.536598 iteration & value:  16  4.71568
2026-01-17 19:44:53.405039 iteration & value:  17  3.8

In [57]:
for i in range(v.shape[0]-1,-1,-1):
    print(str(i).rjust(2),"   ","  ".join(["{:.0f}".format(x).rjust(4) for x in v[i,:]]))

20      556   562   568   573   579   584   588   593   598   602   606   610   613   617   620   623   627   629   632   635   637
19      552   558   564   570   575   581   586   590   595   600   604   608   612   615   619   622   625   629   631   634   636
18      547   554   560   566   572   577   583   588   592   597   602   606   610   614   617   621   624   627   630   633   635
17      542   549   556   562   568   574   579   585   590   594   599   604   608   612   616   619   623   626   629   632   634
16      537   544   551   558   564   570   576   581   587   592   596   601   605   610   614   617   621   624   627   630   633
15      532   539   546   553   560   566   572   578   583   588   593   598   603   607   611   615   619   622   625   628   631
14      527   534   541   548   555   562   568   574   580   585   590   595   600   604   609   613   616   620   623   626   629
13      521   529   536   543   550   557   564   570   576   581   587   59

In [58]:
policy = np.zeros(v.shape)
for state_1 in l_states1:
    for state_2 in l_states2:
        s = np.array([state_1,state_2])
        max_value = -math.inf
        for a in actions:
            if not is_feasible(s, a, capacity):
                continue
            v_s_a = new_value(v, s, a, gamma, capacity, l_prob_demand1, l_prob_demand_2, l_prob_return1, l_prob_return2)
            if v_s_a > max_value:
                max_value, best_action = v_s_a, a
        policy[state_1,state_2] = best_action

for i in range(policy.shape[0]-1,-1,-1):
        print(str(i).rjust(2),"   ","  ".join(["{:.0f}".format(x).rjust(2) for x in policy[i,:]]))

del a
del delta
del best_action
del iter_no
del s
del state_1
del state_2
del v_s_a
del max_value
del i

20      7   6   5   5   4   4   3   3   3   3   2   2   2   2   2   1   1   1   0   0   0
19      7   6   5   4   4   3   3   2   2   2   2   1   1   1   1   1   0   0   0   0   0
18      7   6   5   4   3   3   2   2   1   1   1   1   0   0   0   0   0   0   0   0   0
17      7   6   5   4   3   2   2   1   1   0   0   0   0   0   0   0   0   0   0   0   0
16      6   6   5   4   3   2   1   1   0   0   0   0   0   0   0   0   0   0   0   0   0
15      6   5   5   4   3   2   1   0   0   0   0   0   0   0   0   0   0   0   0   0   0
14      6   5   4   4   3   2   1   0   0   0   0   0   0   0   0   0   0   0   0   0   0
13      6   5   4   3   3   2   1   0   0   0   0   0   0   0   0   0   0   0   0   0   0
12      5   5   4   3   2   2   1   0   0   0   0   0   0   0   0   0   0   0   0   0   0
11      5   4   4   3   2   1   1   0   0   0   0   0   0   0   0   0   0   0   0   0   0
10      4   4   3   3   2   1   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
 9      4 